# 연쇄적 사고 프롬프팅 (Chain-of-Thought Prompting)

## 개념 요약
**연쇄적 사고(Chain-of-Thought, CoT)** 프롬프팅은 대규모 언어 모델(LLM)이 복잡한 추론 문제에 대해 더 정확하고 논리적인 답변을 생성하도록 유도하는 기술임. 이 방법은 모델에게 최종 답변만 요구하는 대신, 문제 해결에 필요한 중간 단계의 **사고 과정(reasoning steps)**을 명시적으로 작성하게 함으로써, 모델이 스스로 논리를 구축하고 검증하도록 이끎.

CoT는 특히 산수 문제,常识 추론, 기호 조작 등 여러 단계를 거쳐야 하는 문제에서 LLM의 성능을 크게 향상시킴. 이는 사람이 복잡한 문제를 풀 때 머릿속으로 생각의 흐름을 전개하거나 종이에 풀이 과정을 적어보는 것과 유사한 원리임.

## 개념적 배경 🧠
CoT의 핵심 아이디어는 모델의 출력 생성 방식을 **'결과 중심'에서 '과정 중심'으로** 바꾸는 데 있음. 기존의 표준 프롬프팅(Standard Prompting)이 `질문 -> 답변`의 직접적인 구조를 따랐다면, CoT 프롬프팅은 `질문 -> 사고 과정 -> 답변`의 구조를 따름.

이러한 접근 방식은 두 가지 주요 이점을 제공함:
1.  **문제 분해(Decomposition):** 모델은 복잡한 문제를 더 작고 관리하기 쉬운 하위 문제들로 나누어 순차적으로 해결할 수 있음.
2.  **해석 가능성(Interpretability):** 모델이 어떤 논리적 경로를 거쳐 결론에 도달했는지 사용자가 확인할 수 있어, 답변의 신뢰도를 평가하고 오류를 수정하기 용이해짐.

CoT는 크게 두 가지 방식으로 구현될 수 있음:
-   **Zero-shot CoT:** 별도의 예시 없이, 프롬프트에 "Let's think step by step." (차근차근 생각해 보자)와 같은 간단한 문구를 추가하여 모델이 스스로 사고 과정을 생성하도록 유도함.
-   **Few-shot CoT:** 몇 가지 `질문-사고과정-답변` 예시(demonstrations)를 프롬프트에 포함하여, 모델이 따라야 할 추론의 형식과 구조를 명확하게 제시함.

## 라이브러리 임포트 (Setup)
이 예제는 특정 라이브러리 없이, 프롬프트 문자열을 구성하는 방식 자체에 초점을 맞춤. LLM API를 사용한다고 가정하고, 프롬프트 구조를 보여주는 데 중점을 둠.

In [1]:
# 특별한 라이브러리 필요 없음
import textwrap # 긴 텍스트 출력을 보기 좋게 정리하기 위해 사용

def print_wrapped(text):
    """콘솔 너비에 맞춰 텍스트를 줄바꿈하여 출력하는 함수"""
    print(textwrap.fill(text, width=80))


## 예제 데이터 준비 (Data Preparation)
모델에게 제시할 복잡한 추론 문제를 정의함. 여기서는 간단한 산수 응용 문제를 사용함.

In [2]:
question = "한 테니스 클럽에 라켓이 5개 있었습니다. 어제 2개의 라켓이 새로 들어왔고, 오늘 회원이 3개의 라켓을 빌려갔습니다. 지금 클럽에 남은 라켓은 총 몇 개인가요?"

print("문제:")
print_wrapped(question)

문제:
한 테니스 클럽에 라켓이 5개 있었습니다. 어제 2개의 라켓이 새로 들어왔고, 오늘 회원이 3개의 라켓을 빌려갔습니다. 지금 클럽에 남은 라켓은
총 몇 개인가요?


## 핵심 로직 구현 (Code Implementation)
표준 프롬프트와 Zero-shot CoT 프롬프트를 각각 구성하여 비교함. 이는 LLM에 전달될 입력 텍스트가 어떻게 다른지 보여줌.

In [3]:
# --- 1. 표준 프롬프팅 (Standard Prompting) --- 
# 모델에게 직접적인 답변을 요구
standard_prompt = f"""Q: {question}
A:"""

print("--- 표준 프롬프트 (Standard Prompt) ---")
print_wrapped(standard_prompt)
print("\n(모델은 이 프롬프트에 대해 바로 숫자 답변을 생성하려고 시도할 것임. 예: '4개')")

# --- 2. 연쇄적 사고 프롬프팅 (Chain-of-Thought Prompting) ---
# 모델에게 단계별 사고를 유도하는 문구 추가 (Zero-shot CoT)
cot_prompt = f"""Q: {question}
A: Let's think step by step.""" # "차근차근 생각해 봅시다." 라는 문구로 CoT 유도

print("\n--- 연쇄적 사고 프롬프트 (Chain-of-Thought Prompt) ---")
print_wrapped(cot_prompt)
print("\n(모델은 이 프롬프트에 대해 먼저 풀이 과정을 생성하고, 그 후에 최종 답변을 내놓을 것임.)")

--- 표준 프롬프트 (Standard Prompt) ---
Q: 한 테니스 클럽에 라켓이 5개 있었습니다. 어제 2개의 라켓이 새로 들어왔고, 오늘 회원이 3개의 라켓을 빌려갔습니다. 지금 클럽에 남은
라켓은 총 몇 개인가요? A:

(모델은 이 프롬프트에 대해 바로 숫자 답변을 생성하려고 시도할 것임. 예: '4개')

--- 연쇄적 사고 프롬프트 (Chain-of-Thought Prompt) ---
Q: 한 테니스 클럽에 라켓이 5개 있었습니다. 어제 2개의 라켓이 새로 들어왔고, 오늘 회원이 3개의 라켓을 빌려갔습니다. 지금 클럽에 남은
라켓은 총 몇 개인가요? A: Let's think step by step.

(모델은 이 프롬프트에 대해 먼저 풀이 과정을 생성하고, 그 후에 최종 답변을 내놓을 것임.)


## 결과 확인 및 시각화 📊
실제 LLM이 각 프롬프트에 대해 생성할 법한 가상의 응답을 비교하여 CoT의 효과를 확인함.

In [4]:
# 가상의 모델 응답 (실제 LLM API 호출 결과라고 가정)

hypothetical_standard_response = "4개"

hypothetical_cot_response = ( 
    "1. 시작할 때 라켓이 5개 있었습니다.\n"
    "2. 새로 2개의 라켓이 들어왔으므로, 총 라켓 수는 5 + 2 = 7개가 됩니다.\n"
    "3. 회원이 3개의 라켓을 빌려갔으므로, 남은 라켓 수는 7 - 3 = 4개가 됩니다.\n"
    "따라서 최종적으로 남은 라켓은 4개입니다."
)

print("--- 표준 프롬프트에 대한 예상 답변 ---")
print_wrapped(f"Q: {question}\nA: {hypothetical_standard_response}")

print("\n--- CoT 프롬프트에 대한 예상 답변 ---")
print(f"Q: {question}\nA: Let's think step by step.\n{hypothetical_cot_response}")

--- 표준 프롬프트에 대한 예상 답변 ---
Q: 한 테니스 클럽에 라켓이 5개 있었습니다. 어제 2개의 라켓이 새로 들어왔고, 오늘 회원이 3개의 라켓을 빌려갔습니다. 지금 클럽에 남은
라켓은 총 몇 개인가요? A: 4개

--- CoT 프롬프트에 대한 예상 답변 ---
Q: 한 테니스 클럽에 라켓이 5개 있었습니다. 어제 2개의 라켓이 새로 들어왔고, 오늘 회원이 3개의 라켓을 빌려갔습니다. 지금 클럽에 남은 라켓은 총 몇 개인가요?
A: Let's think step by step.
1. 시작할 때 라켓이 5개 있었습니다.
2. 새로 2개의 라켓이 들어왔으므로, 총 라켓 수는 5 + 2 = 7개가 됩니다.
3. 회원이 3개의 라켓을 빌려갔으므로, 남은 라켓 수는 7 - 3 = 4개가 됩니다.
따라서 최종적으로 남은 라켓은 4개입니다.


## 고려사항 (Considerations)

* **모델 의존성:** CoT 프롬프팅은 일반적으로 100B 파라미터 이상의 대규모 언어 모델에서 효과적으로 작동하는 것으로 알려져 있음. 작은 모델에서는 CoT를 적용해도 성능 향상이 미미하거나 오히려 성능이 저하될 수 있음.
* **프롬프트 형식:** Zero-shot CoT는 간단하지만, Few-shot CoT처럼 구체적인 예시를 제공하는 것이 더 안정적인 성능을 보장하는 경우가 많음. 문제의 유형과 난이도에 맞는 적절한 CoT 전략을 선택하는 것이 중요함.
* **비용 문제:** 사고 과정을 모두 생성해야 하므로, 표준 프롬프팅에 비해 더 많은 토큰(token)을 사용하게 됨. 이는 API 호출 비용과 응답 시간의 증가로 이어질 수 있음.

## 최종 요약 (Conclusion)

본 노트북은 연쇄적 사고(Chain-of-Thought) 프롬프팅의 개념과 그 효과를 살펴보았음. CoT는 LLM에게 단순히 정답을 요구하는 대신, 문제 해결 과정을 단계별로 서술하도록 유도하는 프롬프팅 기법임.

예제를 통해 표준 프롬프트와 CoT 프롬프트를 비교한 결과, CoT가 어떻게 모델의 추론 과정을 명시적으로 만들어 정확도를 높이고 답변의 신뢰성을 부여하는지 확인할 수 있었음. 복잡하고 논리적인 추론이 필요한 작업에서 CoT는 LLM의 잠재력을 최대한으로 이끌어내는 강력한 도구라고 할 수 있음.